# HW9 - Wightman Paper

In [2]:
# imports
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [9, 6]
matplotlib.rcParams['font.size'] = 11
import numpy as np

This homework focuses on the paper "[Strategies for Low Detection Limit Measurements wih Cyclic Voltammetry](http://pubs.acs.org/doi/abs/10.1021/ac00024a030)" by Mark Wightman's group at North Carolina. After reading the paper, answer the following questions.

## 1
What is the main goal of the research in this paper and why is it important for the Wightman research program?

The main goal is to explore ways to improve the detection limit while simultaneously increasing the S/N ratio. This is relevant to the research program because measuring in vivo catecholamine requires a better sinal to noise than was previously obtained.

## 2
What is Nafion and why is it usefull for science? Why does it improve the selectivity and sensitivity for catecholamines?

Nafion is a perfluorinated ionomer that functions as a cation-exchange membrane. It improves selectivity and sensitivity by filtering out undesired anions such as dihydroxyphenylacetic acid, ascorbate, and uric acid, allowing primarily the catecholamine to pass through and be detected.

## 3
Why do microelectrodes permit rapid cyclic voltammetry?

They allow rapid but uniform changes in voltage that would be hard to achieve using a single, large electrode.

## 4
Why is the background current large when the cyclic voltammetry is done with a fast scan rate?

The background current can be thought of as the gradient between the bulk and the surface; when we scan quickly, the change at the electrode is more rapid, resulting in a larger difference between the bulk and the electrode (less time to diffuse), so we have a large current.

## 5
Can you estimate the background current in their experiment from the information in the paper?

They have already done the digital background subtraction on all of the data they present here, so its value is hard to estimate using just the paper. We are given the scan rate and the area of the electrode, but we need a bit more information to find the capacitance, from which we could then get the background current. However, they state that it is 33nA.

## 6
The maximum gain was 100 nA V<sup>-1</sup>. Where did this value come from? Show how it is calculated. Why couldn't it be made larger?

## 7
What factor limits the scan rate that can be used in the experiments shown in Figure 3? Calculate what the limit should be using the information in the paper.

The increased noise at high band-pass limits the usable scan rate, in this case from the figure it appears that the limit is 12kHz, so using f = 40nv, the scan rate should be limited at 600.

## 8
The authors state that Johnson noise is the fundamental limit to their method. What is Johnson noise? Equation 2 describes it. Provide an intuitive picture for why it depends on temperature, resistance, and frequency.

Johnson noise is random noise from the motion of electrons in the resistor in the i to v transducer. It makes sense that this is dependent on temperature, because an increase in temperature should increase the magnitude of these random motions. The same concept can be applied to frequency; an increased frequency has increased the random motions of the electrons. The relationship to resistance is slightly less intuitive. The higher the resistance, the longer it takes the electrons to move through the area, increasing the amount of random motion we see for each electron.

## 9
Give an intuitive explanation for why the 4 cyclic voltammograms in Figure 3 are different and highlight the important point or points the figure illustrates. What is the purpose of the figure?

As the bandpass is decreased, the voltammograms appear smoother. The main point of this figure is to illustrate the balance between a decreased f (and thus a better S/N), and the over-filtering that can happen for too low f, where the peak is shifted and the amplitude is decreased.

## 10
Page 2968 states the Nyquist criterion is 5 Hz. How did the authors determine that value?

This is just 1 scan/100 ms collection rate times 1/2, to get 5 Hz.

## 11
The authors take data at specific times that are synchronized to the line frequency. Explain why they do this. Explain also why the authors are concerned about variations in the line frequency when they are only changing by 0.003%.

They take these specific times to eliminate the 60 Hz component that is not part of the measured signal. They are concerned because if these times are not exact multiples, the sum and difference frequencies will add a low frequency component to the waveform- essentially they've added more noise artificially!

## 12
On page 2969, the authors state they have a theoretical detection limit of 789 nM. They used a simulation to obtain this value. We have derived expressions for the maximum current for both the rapidly stirred cell where diffusion is not important and the case where it is important. Use one of the equations, the information in the paper, and make your own estimate of the theoretical detection limit using the information in their paper and compare it with their estimate. The diffusion constant for dopamine is 6x10<sup>-6</sup> cm<sup>2</sup> sec<sup>-1</sup>.

In [ ]:
D = 6e-6
#l = nFADC/sqrt(pi*Dt)

## 13
On the same page, the authors find that the slope of the peak current vs the concentration was 0.335 pA nM<sup>-1</sup>. Make your own calculation for what the solpe should be and compare it with their measured slope. State any assumptions or estimates you need to obtain an answer.

## 14
For 5 points extra credit.

The authors used low-pass digital filtering of the data. Implement their technique using the cyclic voltammogram simulation from the previous homework. Use the same conditions for Figure 3A. Add noise to the data like you did in homework 2. Do the Fourier transform like in homework 5. Apodize the transform. Fourier transform back to recreate the voltammogram.

In [5]:
# simulation variables
t = np.linspace(0, 1.75, 3.5e3)
z = np.linspace(0, 0.01, 20)
dt = t[1]-t[0]
dz = z[1]-z[0]

# E(t)
Ei = 0.4  # initial 
v = 1. # rate
E = np.zeros(len(t))
turnaround_index = len(t)/2
E[:turnaround_index] = Ei-v*t[:turnaround_index]  # decending region
E[turnaround_index:] = Ei-v*0.5*t.max()+v*t[:turnaround_index]  # ascending region

F = 96500.
R = 8.314
T = 300.
a = F/(R*T)
n = 1.
E0 = -0.1

# R and O arrays
CR = 1e-5  # initial R concentration
CO = 0.1  # initial O concentration
DR = 1e-5  # R diffusion coefficient 
DO = DR  # O diffusion coefficient
# R and O are concentration arrays in (space, time)
R = np.zeros([len(z), len(t)])
O = np.zeros([len(z), len(t)])
# set concentration in all space for zero time
R[:, 0] = CR
O[:, 0] = CO
# set concentration for all time at furthest distance from electrode
R[-1, :] = CR
O[-1, :] = CO
for j in range(1, len(t)):
    R[0, j] = (1./(1+np.exp(n*a*(E[j]-E0))))*(O[0, j-1]+R[0, j-1])
    O[0, j] = ((np.exp(n*a*(E[j]-E0)))/(1+np.exp(n*a*(E[j]-E0))))*(O[0, j-1]+R[0, j-1])
    for i in range(1, len(z)-1):
        R[i, j] = R[i, j-1]+(dt*DR*(R[i+1, j-1]-2*R[i, j-1]+R[i-1, j-1]))/(dz**2)
        O[i, j] = O[i, j-1]+(dt*DO*(O[i+1, j-1]-2*O[i, j-1]+O[i-1, j-1]))/(dz**2)

# current
area = 1.
I = n*F*area*DO*(O[1]-O[0])/dz
I_noise = np.random.normal(I**2,I)
plt.plot(E, I_noise)
plt.xlim(E.max(), E.min())
plt.xlabel('E')
plt.ylabel('I')
plt.grid()

ValueError: scale <= 0